In [ ]:
import itertools
from gurobipy import Model, GRB
from examples import Examples

def solve_linear_program():
    model = Model("Linear_Programming_Example")
    
    example = Examples()
    ex = example.get_example(7)
    attributes = ex["attributes"]
    relations = ex["relations"]
    fds = ex["fds"]
    
    variables = {}
    for i in range(1, len(attributes) + 1):
        for subset in itertools.combinations(attributes, i):
            name = "".join(subset)
            x = model.addVar(lb=0, ub=len(subset), name=name)
            variables[frozenset(subset)] = x
    
    model.setObjective(variables[frozenset(attributes)], GRB.MAXIMIZE)

    for rel in relations:
        model.addConstr(variables[frozenset(rel)] <= 1, name = "Rel_" + "".join(rel))
    
    for Y, x in fds:
        Yx = Y + x
        model.addConstr(variables[frozenset(Yx)] == variables[frozenset(Y)], "fd_" + "".join(Y) + "->" + "".join(x))
        
        
    # non-Shannon
    if False:
        model.addConstr(variables[frozenset(["A", "B", "C", "D"])] 
                        + variables[frozenset(["A","B","E","F"])] 
                        + variables[frozenset(["C","D","E","F"])] 
                        + variables[frozenset(["A","B","G","H"])] 
                        + variables[frozenset(["C","D","G","H"])] >= 
                        variables[frozenset(["A","B","E","F","C","D"])] 
                        + variables[frozenset(["A","B","G","H","C","D"])] 
                        + variables[frozenset(["E","F","G","H"])] 
                        + variables[frozenset(["A","B"])] 
                        + variables[frozenset(["C","D"])], name="Vamos")
        
    # Shannon inequalities
    if True:
        if True:
            # Monotonicity
            for x in attributes:
                Y = [y for y in attributes if y != x]
                model.addConstr(variables[frozenset(attributes)] - variables[frozenset(Y)] >= 0, name="Monotonicity")
        if True:
            if True:
                # Submodularity
                for j, l in itertools.combinations(attributes, 2):
                    S = [x for x in attributes if x != j and x != l]
                    for i in range(1, len(S) + 1):
                        for subset in itertools.combinations(S, i):
                            subset = list(subset)
                            jS = subset + [j]
                            lS = subset + [l]
                            jlS = subset + [j, l]
                            model.addConstr(variables[frozenset(jS)] + variables[frozenset(lS)] - variables[frozenset(subset)] - variables[frozenset(jlS)] >= 0, name="Submodularity")
            if True:
                # Submodularity for those cases that subset S is empty (this reduces to subadditivity)
                for j, l in itertools.combinations(attributes, 2):
                    model.addConstr(variables[frozenset([j])] + variables[frozenset([l])] - variables[frozenset([j, l])] >= 0, name="Subadditivity")
            
    model.write("Linear_Programming_Example2.lp")
    model.optimize()
    
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found!")
        print(f"Objective value = {model.objVal}")

# Call the function
solve_linear_program()


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]


Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1805 rows, 255 columns and 7161 nonzeros
Model fingerprint: 0x680af297
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 8e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 5 rows and 0 columns
Presolve time: 0.01s
Presolved: 1800 rows, 255 columns, 7156 nonzeros


Optimize a model with 1805 rows, 255 columns and 7161 nonzeros
Model fingerprint: 0x680af297
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 8e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 5 rows and 0 columns
Presolve time: 0.01s
Presolved: 1800 rows, 255 columns, 7156 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+00   1.120000e+02   0.000000e+00      0s
     502    2.0000000e+00   0.000000e+00   0.000000e+00      0s

Sol